# Zadanie 2

In [1]:
library(ggplot2)
library(glue)
library(dplyr)
library(readr)
library(ggthemes)
library(tidyr)

Warning message:
"pakiet 'ggplot2' został zbudowany w wersji R 4.2.2"
Warning message:
"pakiet 'glue' został zbudowany w wersji R 4.2.2"
Warning message:
"pakiet 'dplyr' został zbudowany w wersji R 4.2.2"

Dołączanie pakietu: 'dplyr'


Następujące obiekty zostały zakryte z 'package:stats':

    filter, lag


Następujące obiekty zostały zakryte z 'package:base':

    intersect, setdiff, setequal, union


Warning message:
"pakiet 'readr' został zbudowany w wersji R 4.2.2"
Warning message:
"pakiet 'ggthemes' został zbudowany w wersji R 4.2.2"
Warning message:
"pakiet 'tidyr' został zbudowany w wersji R 4.2.2"


In [2]:
inflation_data <- read.csv("data/EU_Inflation_HICP_data_mod.csv", header = TRUE, sep = ",")
deposits_data <- read.csv("data/EU_deposits_1yr_data_mod.csv", header = TRUE, sep = ",")

In [3]:
head(inflation_data)

,Period,Austria,Belgium,Bulgaria,Cyprus,Czech.Republic,EU..changing.composition.,Germany,Denmark,Estonia,⋯,Luxembourg,Latvia,Malta,Netherlands,Poland,Portugal,Romania,Sweden,Slovenia,Slovakia
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,01/10/2022,11.5,13.1,NA,8.6,NA,NA,11.6,NA,22.4,⋯,8.8,21.8,7.5,16.8,NA,10.6,NA,NA,10.3,14.5
2,01/09/2022,10.9,12.1,15.6,9.0,17.8,10.9,10.9,11.1,24.1,⋯,8.8,22.0,7.4,17.1,15.7,9.8,13.4,10.3,10.6,13.6
3,01/08/2022,9.3,10.5,15.0,9.6,17.1,10.1,8.8,9.9,25.2,⋯,8.6,21.4,7.0,13.7,14.8,9.3,13.3,9.5,11.5,13.4
4,01/07/2022,9.4,10.4,14.9,10.6,17.3,9.8,8.5,9.6,23.2,⋯,9.3,21.3,6.8,11.6,14.2,9.4,13.0,8.3,11.7,12.8
5,01/06/2022,8.7,10.5,14.8,9.0,16.6,9.6,8.2,9.1,22.0,⋯,10.3,19.2,6.1,9.9,14.2,9.0,13.0,8.9,10.8,12.6
6,01/05/2022,7.7,9.9,13.4,8.8,15.2,8.8,8.7,8.2,20.1,⋯,9.1,16.8,5.8,10.2,12.8,8.1,12.4,7.5,8.7,11.8


In [4]:
head(deposits_data)

,Period,Austria..Euro,Belgium..Euro,Bulgaria..Bulgarian.lev,Cyprus..Euro,Czech.Republic..Czech.koruna,Germany..Euro,Denmark..Danish.krone,Estonia..Estonian.kroon,Estonia..Euro,⋯,Malta..Maltese.lira,Netherlands..Euro,Poland..Polish.zloty,Portugal..Euro,Romania..Romanian.leu,Sweden..Swedish.krona,Slovenia..Euro,Slovenia..Slovenian.tolar,Slovakia..Euro,Slovakia..Slovak.koruna
,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,⋯,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>
1,01/09/2022,0.80,0.84,0.12,0.11,6.10,0.84,1.15,NA,0.94,⋯,,1.84,5.44,0.05,6.44,1.65,0.11,NA,0.77,NA
2,01/08/2022,0.37,0.37,0.11,0.06,5.98,0.49,0.25,NA,0.83,⋯,,1.53,5.22,0.07,6.22,1.21,0.04,NA,0.48,NA
3,01/07/2022,0.23,0.11,0.09,0.05,5.95,0.31,-0.44,NA,0.59,⋯,,1.59,4.85,0.09,5.49,0.90,0.03,NA,0.21,NA
4,01/06/2022,0.12,0.14,0.06,0.05,5.37,0.17,-0.45,NA,0.49,⋯,,1.47,4.50,0.07,4.37,0.53,0.03,NA,0.16,NA
5,01/05/2022,0.08,0.07,0.04,0.05,4.53,0.14,-0.5,NA,0.50,⋯,,1.23,3.38,0.04,3.21,0.38,0.04,NA,0.21,NA
6,01/04/2022,0.10,0.06,0.04,0.05,4.17,0.14,-0.51,NA,0.44,⋯,,1.15,2.03,0.04,2.56,0.32,0.03,NA,0.24,NA


In [5]:
inflation_data$Period <- as.Date(inflation_data$Period, format = "%d/%m/%Y")
deposits_data$Period <- as.Date(deposits_data$Period, format = "%d/%m/%Y")

In [6]:
deposits_data[, 2:34] <- sapply(deposits_data[, 2:34], as.numeric)
# str(deposits_data)

Warning message in lapply(X = X, FUN = FUN, ...):
"pojawiły się wartości NA na skutek przekształcenia"
Warning message in lapply(X = X, FUN = FUN, ...):
"pojawiły się wartości NA na skutek przekształcenia"
Warning message in lapply(X = X, FUN = FUN, ...):
"pojawiły się wartości NA na skutek przekształcenia"
Warning message in lapply(X = X, FUN = FUN, ...):
"pojawiły się wartości NA na skutek przekształcenia"
Warning message in lapply(X = X, FUN = FUN, ...):
"pojawiły się wartości NA na skutek przekształcenia"
Warning message in lapply(X = X, FUN = FUN, ...):
"pojawiły się wartości NA na skutek przekształcenia"
Warning message in lapply(X = X, FUN = FUN, ...):
"pojawiły się wartości NA na skutek przekształcenia"


In [7]:
eurozone_countries_inflation <- select(inflation_data, matches("Period|Austria|Belgium|Cyprus|Germany|Estonia|Spain|Finland|France|Greece|Ireland|Italy|Lithuania|Luxembourg|Latvia|Malta|Netherlands|Portugal|Slovenia|Slovakia"))
other_countries_inflation <- select(inflation_data, !matches("EU|Austria|Belgium|Cyprus|Germany|Estonia|Spain|Finland|France|Greece|Ireland|Italy|Lithuania|Luxembourg|Latvia|Malta|Netherlands|Portugal|Slovenia|Slovakia"))
# EU_countries_inflation <- select(inflation_data, matches("Period|EU"))
# Poland_inflation <- select(inflation_data, matches("Period|Poland"))

In [8]:
head(eurozone_countries_inflation)

,Period,Austria,Belgium,Cyprus,Germany,Estonia,Spain,Finland,France,Greece,Ireland,Italy,Lithuania,Luxembourg,Latvia,Malta,Netherlands,Portugal,Slovenia,Slovakia
,<date>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,2022-10-01,11.5,13.1,8.6,11.6,22.4,7.3,8.3,7.1,9.8,9.6,12.8,22.0,8.8,21.8,7.5,16.8,10.6,10.3,14.5
2,2022-09-01,10.9,12.1,9.0,10.9,24.1,9.0,8.4,6.2,12.1,8.6,9.4,22.5,8.8,22.0,7.4,17.1,9.8,10.6,13.6
3,2022-08-01,9.3,10.5,9.6,8.8,25.2,10.5,7.9,6.6,11.2,9.0,9.1,21.1,8.6,21.4,7.0,13.7,9.3,11.5,13.4
4,2022-07-01,9.4,10.4,10.6,8.5,23.2,10.7,8.0,6.8,11.3,9.6,8.4,20.9,9.3,21.3,6.8,11.6,9.4,11.7,12.8
5,2022-06-01,8.7,10.5,9.0,8.2,22.0,10.0,8.1,6.5,11.6,9.6,8.5,20.5,10.3,19.2,6.1,9.9,9.0,10.8,12.6
6,2022-05-01,7.7,9.9,8.8,8.7,20.1,8.5,7.1,5.8,10.5,8.3,7.3,18.5,9.1,16.8,5.8,10.2,8.1,8.7,11.8


In [9]:
eurozone_countries_inflation_avg <- data.frame(eurozone_countries_inflation[1], (rowMeans(eurozone_countries_inflation[2:20], na.rm=TRUE)))
other_countries_inflation_avg <- data.frame(other_countries_inflation[1], (rowMeans(other_countries_inflation[2:9], na.rm=TRUE)))
EU_countries_inflation <- data.frame(eurozone_countries_inflation[1], select(inflation_data, matches("EU")))
Poland_inflation <- data.frame(eurozone_countries_inflation[1], select(inflation_data, matches("Poland")))

In [10]:
inflation_avg <- data.frame(eurozone_countries_inflation[1], eurozone_countries_inflation_avg[2], other_countries_inflation_avg[2], EU_countries_inflation[2], Poland_inflation[2])

colnames(inflation_avg)[1] <- "Period"
colnames(inflation_avg)[2] <- "Eurozone"
colnames(inflation_avg)[3] <- "Other"
colnames(inflation_avg)[4] <- "EU"
colnames(inflation_avg)[5] <- "Poland"

In [11]:
# January 2018 - October 2022
n_months = 58

In [12]:
copy_inf = subset(inflation_data, select=c(Period))[c(2:n_months),]

In [13]:
inflation_from2018 = subset(inflation_data, select=-c(EU..changing.composition.))[c(2:n_months),]
inflation_from2018['average'] = rowMeans(inflation_from2018[2:28], na.rm=TRUE)

In [14]:
deposits_from2018 = subset(deposits_data, select=-c(Slovakia..Slovak.koruna, Slovenia..Slovenian.tolar, Malta..Maltese.lira, Estonia..Estonian.kroon, Latvia..Latvian.lats, Lithuania..Lithuanian.litas))[c(1:(n_months-1)),]
deposits_from2018['average'] = rowMeans(deposits_from2018[2:28], na.rm=TRUE)

In [15]:
inf_cols = colnames(subset(inflation_data, select=-c(Period, EU..changing.composition.)))

In [16]:
dep_cols = colnames(subset(deposits_data[2:34], select=-c(Slovakia..Slovak.koruna, Slovenia..Slovenian.tolar, Malta..Maltese.lira, Estonia..Estonian.kroon, Latvia..Latvian.lats, Lithuania..Lithuanian.litas)))

In [17]:
inf_cols

[1] "Austria"        "Belgium"        "Bulgaria"       "Cyprus"        
 [5] "Czech.Republic" "Germany"        "Denmark"        "Estonia"       
 [9] "Spain"          "Finland"        "France"         "Greece"        
[13] "Croatia"        "Hungary"        "Ireland"        "Italy"         
[17] "Lithuania"      "Luxembourg"     "Latvia"         "Malta"         
[21] "Netherlands"    "Poland"         "Portugal"       "Romania"       
[25] "Sweden"         "Slovenia"       "Slovakia"

In [18]:
dep_cols

[1] "Austria..Euro"                "Belgium..Euro"               
 [3] "Bulgaria..Bulgarian.lev"      "Cyprus..Euro"                
 [5] "Czech.Republic..Czech.koruna" "Germany..Euro"               
 [7] "Denmark..Danish.krone"        "Estonia..Euro"               
 [9] "Spain..Euro"                  "Finland..Euro"               
[11] "France..Euro"                 "Greece..Euro"                
[13] "Croatia..Croatian.kuna"       "Hungary..Hungarian.forint"   
[15] "Ireland..Euro"                "Italy..Euro"                 
[17] "Lithuania..Euro"              "Luxembourg..Euro"            
[19] "Latvia..Euro"                 "Malta..Euro"                 
[21] "Netherlands..Euro"            "Poland..Polish.zloty"        
[23] "Portugal..Euro"               "Romania..Romanian.leu"       
[25] "Sweden..Swedish.krona"        "Slovenia..Euro"              
[27] "Slovakia..Euro"

In [19]:
deposits_to_inflation <- data.frame(copy_inf)
for (i in 1:length(dep_cols)){
    col_inf <- inf_cols[i]
    col_dep <- dep_cols[i]
    new_column = deposits_from2018[col_dep] / inflation_from2018[col_inf]
    deposits_to_inflation[col_inf] = new_column
}
names(deposits_to_inflation)[names(deposits_to_inflation) == "copy_inf"] <- "Period"
# deposits_to_inflation[Period] = periods_from2018
deposits_to_inflation['average'] = rowMeans(deposits_from2018[2:28], na.rm=TRUE)

In [24]:
# eurozone
deposits_to_inflation['average_eurozone'] = 
rowMeans(select(deposits_to_inflation, matches(
    "Austria|Belgium|Cyprus|Germany|Estonia|Spain|Finland|France|Greece|Ireland|Italy|Lithuania|Luxembourg|Latvia|Malta|Netherlands|Portugal|Slovenia|Slovakia")), na.rm=TRUE)

In [28]:
select(deposits_to_inflation, matches(
    "Austria|Belgium|Cyprus|Germany|Estonia|Spain|Finland|France|Greece|Ireland|Italy|Lithuania|Luxembourg|Latvia|Malta|Netherlands|Portugal|Slovenia|Slovakia"))

Austria,Belgium,Cyprus,Germany,Estonia,Spain,Finland,France,Greece,Ireland,Italy,Lithuania,Luxembourg,Latvia,Malta,Netherlands,Portugal,Slovenia,Slovakia
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0.07339450,0.069421488,0.012222222,0.077064220,0.03900415,0.017777778,0.14285714,0.18064516,0.011570248,0.020930233,0.12340426,0.011111111,0.082954545,0.028181818,0.07972973,0.10760234,0.005102041,0.010377358,0.05661765
0.03978495,0.035238095,0.006250000,0.055681818,0.03293651,0.000952381,0.07848101,0.12878788,0.009821429,0.006666667,0.09670330,0.024644550,0.083720930,0.019158879,0.07714286,0.11167883,0.007526882,0.003478261,0.03582090
0.02446809,0.010576923,0.004716981,0.036470588,0.02543103,0.002803738,0.04000000,0.09411765,0.011504425,0.011458333,0.07857143,0.020574163,0.016129032,0.005633803,0.07058824,0.13706897,0.009574468,0.002564103,0.01640625
0.01379310,0.013333333,0.005555556,0.020731707,0.02227273,0.001000000,0.03456790,0.06769231,0.010344828,0.002083333,0.04588235,0.009756098,0.010679612,0.008333333,0.06393443,0.14848485,0.007777778,0.002777778,0.01269841
0.01038961,0.007070707,0.005681818,0.016091954,0.02487562,0.001176471,0.02253521,0.07068966,0.011428571,0.002409639,0.06438356,0.010810811,0.010989011,0.009523810,0.06551724,0.12058824,0.004938272,0.004597701,0.01779661
0.01408451,0.006451613,0.005813953,0.017948718,0.02303665,0.001204819,0.03793103,0.07777778,0.014285714,0.002739726,0.06984127,0.007831325,0.004444444,0.010687023,0.07407407,0.10267857,0.005405405,0.004054054,0.02201835
0.01212121,0.007526882,0.008064516,0.015789474,0.03175676,0.001020408,0.03103448,0.09411765,0.016250000,0.015942029,0.09117647,0.015384615,0.016455696,0.028695652,0.09555556,0.09230769,0.007272727,0.005000000,0.02500000
0.01636364,0.007368421,0.012068966,0.010909091,0.03534483,0.001315789,0.04772727,0.11666667,0.022222222,0.022807018,0.10322581,0.002857143,-0.050000000,0.036363636,0.10000000,0.13698630,0.009090909,0.007142857,0.04337349
0.02000000,0.017647059,0.012000000,0.021568627,0.04090909,0.001612903,0.04390244,0.14545455,0.025454545,0.026000000,0.09803922,0.004065041,-0.034782609,0.033333333,0.09756098,0.12368421,0.011764706,0.006666667,0.03116883


In [25]:
# and non-eurozone
# deposits_to_inflation['average_not_eurozone'] = 
rowMeans(select(deposits_to_inflation, !matches(
    "Austria|Belgium|Cyprus|Germany|Estonia|Spain|Finland|France|Greece|Ireland|Italy|Lithuania|Luxembourg|Latvia|Malta|Netherlands|Portugal|Slovenia|Slovakia")), na.rm=TRUE)

ERROR: Error in rowMeans(select(deposits_to_inflation, !matches("Austria|Belgium|Cyprus|Germany|Estonia|Spain|Finland|France|Greece|Ireland|Italy|Lithuania|Luxembourg|Latvia|Malta|Netherlands|Portugal|Slovenia|Slovakia")), : argument 'x' musi być liczbą


In [26]:
deposits_to_inflation

Period,Austria,Belgium,Bulgaria,Cyprus,Czech.Republic,Germany,Denmark,Estonia,Spain,⋯,Malta,Netherlands,Poland,Portugal,Romania,Sweden,Slovenia,Slovakia,average,average_eurozone
<date>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2022-09-01,0.07339450,0.069421488,0.007692308,0.012222222,0.34269663,0.077064220,0.10360360,0.03900415,0.017777778,⋯,0.07972973,0.10760234,0.34649682,0.005102041,0.4805970,0.16019417,0.010377358,0.05661765,1.2850000,0.06052463
2022-08-01,0.03978495,0.035238095,0.007333333,0.006250000,0.34970760,0.055681818,0.02525253,0.03293651,0.000952381,⋯,0.07714286,0.11167883,0.35270270,0.007526882,0.4676692,0.12736842,0.003478261,0.03582090,1.3492593,0.04497243
2022-07-01,0.02446809,0.010576923,0.006040268,0.004716981,0.34393064,0.036470588,-0.04583333,0.02543103,0.002803738,⋯,0.07058824,0.13706897,0.34154930,0.009574468,0.4223077,0.10843373,0.002564103,0.01640625,1.1159259,0.03256096
2022-06-01,0.01379310,0.013333333,0.004054054,0.005555556,0.32349398,0.020731707,-0.04945055,0.02227273,0.001000000,⋯,0.06393443,0.14848485,0.31690141,0.007777778,0.3361538,0.05955056,0.002777778,0.01269841,0.8903704,0.02640523
2022-05-01,0.01038961,0.007070707,0.002985075,0.005681818,0.29802632,0.016091954,-0.06097561,0.02487562,0.001176471,⋯,0.06551724,0.12058824,0.26406250,0.004938272,0.2588710,0.05066667,0.004597701,0.01779661,0.7237037,0.02534182
2022-04-01,0.01408451,0.006451613,0.003305785,0.005813953,0.31590909,0.017948718,-0.06891892,0.02303665,0.001204819,⋯,0.07407407,0.10267857,0.17807018,0.005405405,0.2188034,0.04848485,0.004054054,0.02201835,0.6200000,0.02643732
2022-03-01,0.01212121,0.007526882,0.003809524,0.008064516,0.29495798,0.015789474,-0.09000000,0.03175676,0.001020408,⋯,0.09555556,0.09230769,0.14509804,0.007272727,0.2135417,0.03968254,0.005000000,0.02500000,0.5618519,0.03213010
2022-02-01,0.01636364,0.007368421,0.004761905,0.012068966,0.34700000,0.010909091,-0.11132075,0.03534483,0.001315789,⋯,0.10000000,0.13698630,0.14814815,0.009090909,0.1936709,0.04318182,0.007142857,0.04337349,0.4574074,0.03588600
2022-01-01,0.02000000,0.017647059,0.005194805,0.012000000,0.31704545,0.021568627,-0.11632653,0.04090909,0.001612903,⋯,0.09756098,0.12368421,0.11379310,0.011764706,0.1958333,0.05641026,0.006666667,0.03116883,0.4155556,0.03821314


In [ ]:
country1 = 'Poland'
country2 = 'France'
country3 = 'Hungary'
country4 = 'Slovakia'

In [ ]:
ggplot(data = deposits_to_inflation) +
  geom_line(aes(x = Period, y = Poland), color="#FF0000", size=1) +
  theme(plot.margin = margin(.2,1,0.2,0.2, "cm")) + 
  labs(x = "Date",
       y = "Deposits/Inflation",
       title = "Deposits/Inflation in Poland")

In [ ]:
plot <- function(dataset, x_axis, y1, y2, y3, y4, y5, y_lab, title) {
  ggplot(data = dataset) +
    geom_line(aes(x = x_axis, y = y1), colour="#FF0000", size=1) +
    geom_line(aes(x = x_axis, y = y2), color="#0072BD", size=1) +
    geom_line(aes(x = x_axis, y = y3), color="#00FF00", size=1) +
    geom_line(aes(x = x_axis, y = y4), color="#000000", size=1) +
    geom_line(aes(x = x_axis, y = y5), color="#8B008B", size=1) +
    theme(plot.margin = margin(.5,.5,.5,.5, "cm")) + 
    scale_x_date(date_labels = "%b-%Y") +
    labs(x = "Date",
        y = y_lab, 
        title = title)
}

In [ ]:
plot(deposits_to_inflation, deposits_to_inflation$Period, deposits_to_inflation$Poland, deposits_to_inflation$France, 
     deposits_to_inflation$Hungary, deposits_to_inflation$Romania, deposits_to_inflation$average, "Deposits/Inflation", "Deposits/Inflation in chosen countries")

In [ ]:
plot(inflation_from2018, inflation_from2018$Period, inflation_from2018$Poland, inflation_from2018$France, 
     inflation_from2018$Hungary, inflation_from2018$Romania, inflation_from2018$average, "Inflation", "Inflation in chosen countries")

In [ ]:
plot(deposits_from2018, deposits_from2018$Period, deposits_from2018$Poland, deposits_from2018$France, 
     deposits_from2018$Hungary, deposits_from2018$Romania, deposits_from2018$average, "Deposits", "Deposits in chosen countries")

In [ ]:
# Creating ranges of intervals
before_pandemic <- as.Date("01/01/2018", "%d/%m/%Y")
pandemic <- as.Date("01/03/2020", "%d/%m/%Y")
before_war <- as.Date("01/02/2021", "%d/%m/%Y")
war <- as.Date("01/02/2022", "%d/%m/%Y")
now <- as.Date("01/09/2022", "%d/%m/%Y")

In [ ]:
# Split data to intervals
before_pandemic_data <- deposits_to_inflation %>% filter(Period >= before_pandemic & Period < pandemic)
pandemic_data <- deposits_to_inflation %>% filter(Period >= pandemic & Period < before_war)
before_war_data <- deposits_to_inflation %>% filter(Period >= before_war & Period < war)
war_data <- deposits_to_inflation %>% filter(Period >= war & Period <= now)

In [ ]:
# before pandemic
plot(before_pandemic_data, before_pandemic_data$Period, before_pandemic_data$Poland, before_pandemic_data$France, 
     before_pandemic_data$Hungary, before_pandemic_data$Romania, before_pandemic_data$average, "Deposits/Inflation", "Before pandemic")

In [ ]:
# during pandemic
plot(pandemic_data, pandemic_data$Period, pandemic_data$Poland, pandemic_data$France, 
     pandemic_data$Hungary, pandemic_data$Romania, pandemic_data$average, "Deposits/Inflation", "During pandemic")

In [ ]:
plot(before_war_data, before_war_data$Period, before_war_data$Poland, before_war_data$France, 
     before_war_data$Hungary, before_war_data$Romania, before_war_data$average, "Deposits/Inflation", "Year before war started")

In [ ]:
plot(war_data, war_data$Period, war_data$Poland, war_data$France, war_data$Hungary, war_data$Romania, war_data$average, "Deposits/Inflation", "After war started")